In [233]:
import pandas as pd    #importing libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.svm import LinearSVC
#from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFECV


In [234]:
warnings.filterwarnings('ignore') #handle warnings

In [235]:
#Step 1.Loading the benchmark datasets and visualize the data and test for null or missing data
training_dataset = pd.read_csv('KDDTrain+_txt.csv', sep=',') #loading the training datasets and visualizing its dimensions
training_dataset.shape

(125972, 43)

In [236]:
testing_dataset = pd.read_csv('KDDTest+_txt.csv', sep=',') #loading the testing datasets and print its dimensions
testing_dataset.shape

(22543, 43)

In [237]:
#columns header names from the benchmark open source dataset NSL-KDD 
header_name = ["duration","protocol_type","service","flag","src_bytes",    
               "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
               "logged_in","num_compromised","root_shell","su_attempted","num_root",
               "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
               "is_host_login","is_guest_login","count","srv_count","serror_rate",
               "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
               "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
               "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
               "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
               "dst_host_rerror_rate","dst_host_srv_rerror_rate","class","data"]

In [238]:
#alocate the attribute name to database and visualize the dataset
training_dataset = pd.read_csv("KDDTrain+_txt.csv", header=None, names = header_name) 
#training_dataset

In [239]:
testing_dataset = pd.read_csv("KDDTest+_txt.csv", header=None, names = header_name)
#testing_dataset

In [240]:
#testing for duplicate data in the datasets
duplicate = training_dataset.duplicated().sum()
duplicate

0

In [241]:
duplicate1 = testing_dataset.duplicated().sum()
duplicate1

0

In [242]:
training_dataset.describe(include='object') #indetifying the categorical data to binary encode them in the training dataset
#total number of categorical features are:3+70+11=84 (protoco_type, service, flag)

,protocol_type,service,flag,class
count,125973,125973,125973,125973
unique,3,70,11,23
top,tcp,http,SF,normal
freq,102689,40338,74945,67343


In [243]:
#Idetifying the categorical data to binary encode them in the testing dataset(protocol_type, service, flag)
testing_dataset.describe(include='object')
#total numbers of categorical features are 3+64+11=78, 
#there is a difference in category data (service) between the testing datasets and training datasets we add empty columns.

,protocol_type,service,flag,class
count,22544,22544,22544,22544
unique,3,64,11,38
top,tcp,http,SF,normal
freq,18880,7853,14875,9711


In [244]:
print('Distribution of categories in service:') #identifying the distribution of the categorical data into the testing data set
print(training_dataset['service'].value_counts())


Distribution of categories in service:
service
http         40338
private      21853
domain_u      9043
smtp          7313
ftp_data      6860
             ...  
tftp_u           3
http_8001        2
aol              2
harvest          2
http_2784        1
Name: count, Length: 70, dtype: int64


In [245]:
print('Attack type labels distribution in the training dataset:')
print(training_dataset['class'].value_counts())

Attack type labels distribution in the training dataset:
class
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: count, dtype: int64


In [246]:
print('Attack type label distribution in the testing dataset:')
print(testing_dataset['service'].value_counts())

Attack type label distribution in the testing dataset:
service
http       7853
private    4774
telnet     1626
pop_3      1019
smtp        934
           ... 
printer      11
ntp_u        10
rje           8
tim_i         6
tftp_u        1
Name: count, Length: 64, dtype: int64


In [247]:
print(testing_dataset['service'].value_counts())

service
http       7853
private    4774
telnet     1626
pop_3      1019
smtp        934
           ... 
printer      11
ntp_u        10
rje           8
tim_i         6
tftp_u        1
Name: count, Length: 64, dtype: int64


In [248]:
#Creating header for the dummy columns by adding together the categorical data type with their values(e.g. Protocol have the 3 values icmp, tcp, udp)
#Training dataset
protocol_type=sorted(training_dataset.protocol_type.unique()) #sorting the unique values of the array
name = 'Protocol_'
protocol_type1=[name+ p for p in protocol_type] 

service_type=sorted(training_dataset.service.unique())
name1 = 'service_type_'
service_type1=[name1 + p for p in service_type]

flag_type=sorted(training_dataset.flag.unique())
name2 = 'flag_type_'
flag_type1=[name2 + p for p in flag_type]
dummy_columns = protocol_type1 + service_type1 + flag_type1 #concatenete the lists

#Testing dataset
protocol_type_test=sorted(testing_dataset.protocol_type.unique())
name3 = 'Protocol_'
protocol_type_test_1=[name3+ p for p in protocol_type_test] 

service_type_test=sorted(testing_dataset.service.unique())
name4 = 'service_type_'
service_type_test_1=[name4 + p for p in service_type_test]

flag_type_test=sorted(testing_dataset.flag.unique())
name5 = 'flag_type_'
flag_type_test_1=[name5 + p for p in flag_type_test]
dummy_columns_test = protocol_type_test_1 + service_type_test_1 + flag_type_test_1



In [249]:
#Label encoding - transforming categorical features into numbers
#Training and testing dataset
categ_col = ['protocol_type', 'service', 'flag']
categorical_val_train = training_dataset[categ_col]
train_label = categorical_val_train.apply(LabelEncoder().fit_transform)
categorical_val_test = testing_dataset[categ_col]
test_label = categorical_val_test.apply(LabelEncoder().fit_transform)


In [250]:
#One-Hot Encoding the data from training and testing database to be used in the machine learning algorithm
one_hot_enc = OneHotEncoder(handle_unknown='ignore') #creating instance of the one hot encoding class
ohe_training = one_hot_enc.fit_transform(train_label)
ohe_testing = one_hot_enc.fit_transform(test_label)
new_training = pd.DataFrame(ohe_training.toarray(), columns= dummy_columns)
new_testing = pd.DataFrame(ohe_testing.toarray(), columns = dummy_columns_test)
 

In [251]:
train_serv = training_dataset['service'].tolist()  
test_serv = testing_dataset['service'].tolist()
diff = list(set(train_serv) - set(test_serv)) #determining the difference in 'service' feature between the two databases
feature = 'service_'
diff = [feature + p for p in diff]   
for column in diff:     #adding columns of zero to balance the database
    new_testing[column] = 0
   

In [252]:
#Adding together the numerical data frames with the categorical data frame for both databases
train_dbs_new = training_dataset.join(new_training)
train_dbs_new.drop('protocol_type', axis = 1, inplace = True)
train_dbs_new.drop('flag', axis = 1, inplace = True)
train_dbs_new.drop('service', axis = 1, inplace =True)

test_dbs_new = testing_dataset.join(new_testing)
test_dbs_new.drop('protocol_type', axis = 1, inplace =True)
test_dbs_new.drop('flag', axis = 1, inplace = True)
test_dbs_new.drop('service', axis = 1, inplace =True)
print(train_dbs_new.shape)
print(test_dbs_new.shape)

(125973, 124)
(22544, 124)


In [253]:
#changing the names of the attack types labels with the new encoding as follows in the both datasets: 
#1 - normal, 2 -denial of service, 3-probing attack, 4-route to local- R2L, 5-user to root attack - U2R
db_training_new = train_dbs_new['class'].replace({ 'normal' : 1, 'neptune' : 2 ,'back': 2, 'land': 2, 'pod': 2, 'smurf': 2, 'teardrop': 2,'mailbomb': 2, 'apache2': 2, 'processtable': 2, 'udpstorm': 2, 'worm': 2,
                                                    'ipsweep' : 3,'nmap' : 3,'portsweep' : 3,'satan' : 3,'mscan' : 3,'saint' : 3
                                                    ,'ftp_write': 4,'guess_passwd': 4,'imap': 4,'multihop': 4,'phf': 4,'spy': 4,'warezclient': 4,'warezmaster': 4,'sendmail': 4,'named': 4,'snmpgetattack': 4,'snmpguess': 4,'xlock': 4,'xsnoop': 4,'httptunnel': 4,
                                                    'buffer_overflow': 5,'loadmodule': 5,'perl': 5,'rootkit': 5,'ps': 5,'sqlattack': 5,'xterm': 5})
db_testing_new = test_dbs_new['class'].replace({ 'normal' : 1, 'neptune' : 2 ,'back': 2, 'land': 2, 'pod': 2, 'smurf': 2, 'teardrop': 2,'mailbomb': 2, 'apache2': 2, 'processtable': 2, 'udpstorm': 2, 'worm': 2,
                                                'ipsweep' : 3,'nmap' : 3,'portsweep' : 3,'satan' : 3,'mscan' : 3,'saint' : 3
                                                ,'ftp_write': 4,'guess_passwd': 4,'imap': 4,'multihop': 4,'phf': 4,'spy': 4,'warezclient': 4,'warezmaster': 4,'sendmail': 4,'named': 4,'snmpgetattack': 4,'snmpguess': 4,'xlock': 4,'xsnoop': 4,'httptunnel': 4,
                                                 'buffer_overflow': 5,'loadmodule': 5,'perl': 5,'rootkit': 5,'ps': 5,'sqlattack': 5,'xterm': 5})
train_dbs_new['class'] = db_training_new
test_dbs_new['class'] = db_testing_new

In [254]:
DOS_encoding = [3, 4, 5];Probe_encoding = [2, 4, 5]; R2L_encoding = [2, 3, 5];U2R_encoding = [2, 3, 4]

denialOfService_train = train_dbs_new[~train_dbs_new['class'].isin(DOS_encoding)]
Probing_train = train_dbs_new[~train_dbs_new['class'].isin(Probe_encoding)]
Root2Local_train = train_dbs_new[~train_dbs_new['class'].isin(R2L_encoding)]
User2Root_train = train_dbs_new[~train_dbs_new['class'].isin(U2R_encoding)]

Denial_test = test_dbs_new[~test_dbs_new['class'].isin(DOS_encoding)]
Probing_test = test_dbs_new[~test_dbs_new['class'].isin(Probe_encoding)]
Root2Local_test =  test_dbs_new[~test_dbs_new['class'].isin(R2L_encoding)]
User2Root_test =  test_dbs_new[~test_dbs_new['class'].isin(U2R_encoding)]

print('Number of records per attack type in training database\n')
print('Numbers of records of denial of service attacks', denialOfService_train.shape)
print('Numbers of records of probing attacks', Probing_train.shape)
print('Numbers of records of root to local attacks', Root2Local_train.shape)
print('Numbers of records of user to root attacks', User2Root_train.shape)
print()
print('Number of records per attack type in testing database\n')
print('Numbers of records of denial of service attacks', Denial_test.shape)
print('Numbers of records of probing attacks', Probing_test.shape)
print('Numbers of records of root to local attacks', Root2Local_test.shape)
print('Numbers of records of user to root attacks', User2Root_test.shape)


Number of records per attack type in training database

Numbers of records of denial of service attacks (113270, 124)
Numbers of records of probing attacks (78999, 124)
Numbers of records of root to local attacks (68338, 124)
Numbers of records of user to root attacks (67395, 124)

Number of records per attack type in testing database

Numbers of records of denial of service attacks (17171, 124)
Numbers of records of probing attacks (12132, 124)
Numbers of records of root to local attacks (12596, 124)
Numbers of records of user to root attacks (9778, 124)


In [255]:
#Spliting the data according to its features(X) and Y as result (outcome) variables data frame
#Trainig database
X_train_dos = denialOfService_train.drop('class',axis=1)
y_train_dos = denialOfService_train[['class']]

X_train_Probing = Probing_train.drop('class',axis=1)
y_train_Probing = Probing_train[['class']]

X_train_r2l = Root2Local_train.drop('class',axis=1)
y_train_r2l = Root2Local_train[['class']]

X_train_u2r = User2Root_train.drop('class',axis=1)
y_train_u2r = User2Root_train[['class']]
#Testing database
X_test_dos = Denial_test.drop('class',axis=1)
y_test_dos = Denial_test[['class']]

X_probe_test = Probing_test.drop('class',axis=1)
y_probe_test = Probing_test[['class']]

X_test_r2l = Root2Local_test.drop('class',axis=1)
y_test_r2l = Root2Local_test[['class']]

X_test_u2r = User2Root_test.drop('class',axis=1)
y_test_u2r = User2Root_test[['class']]


In [256]:
colNames=list(X_train_dos)
colNames_test=list(X_test_dos)

In [257]:
# Using standard scaler to resize the values' distribution- to avoid biases
# Training datasets 
X_scaler_dos = preprocessing.StandardScaler().fit_transform(X_train_dos)
X_scaler_Probe = preprocessing.StandardScaler().fit_transform(X_train_Probing)
X_scaler_r2l = preprocessing.StandardScaler().fit_transform(X_train_r2l)
X_scaler_u2r = preprocessing.StandardScaler().fit_transform(X_train_u2r)
#Testing database
X_test_scaler_dos = preprocessing.StandardScaler().fit_transform(X_test_dos)
X_test_scaler_probe = preprocessing.StandardScaler().fit_transform(X_probe_test)
X_test_scaler_r2l = preprocessing.StandardScaler().fit_transform(X_test_r2l)
X_test_scaler_u2r = preprocessing.StandardScaler().fit_transform(X_test_u2r)


In [258]:
#Feature selection for training database by  using Univariate Feature Selection method
#(Input data are numerical; Output data → Categorical data )
#To extract the most representative features from the dataset, based on highest score, we use SelectKBest model function)
########################################################################
from sklearn.feature_selection import RFE
selector = SelectKBest(f_classif, k=10)  # k-is the best 10 features percentage wise from datasets

#DOS
variable = selector.fit(X_scaler_dos, y_train_dos)
support = variable.get_support()
new_DoS=[j for j, p in enumerate (support) if p]   #printing the 10 best features from DOS database
new_DoS=list( colNames[j] for j in new_DoS)


#Probing
variable1= selector.fit(X_scaler_Probe, y_train_Probing)
support1 = variable.get_support()
new_probing=[j for j, p in enumerate (support1) if p]  #printing the 10 best features from Probing database
new_probing=list( colNames[j] for j in new_probing)
#R2L
variable2= selector.fit(X_scaler_r2l, y_train_r2l)
support2 = variable.get_support()
new_r2l=[j for j, p in enumerate (support2) if p]   #printing the 10 best features from R2L database
new_r2l=list( colNames[j] for j in new_r2l)
#U2R
variable3= selector.fit(X_scaler_u2r, y_train_u2r)
support3 = variable.get_support()
new_u2r=[j for j, p in enumerate (support3) if p]   #printing the 10 best features from U2R database
new_u2r=list( colNames[j] for j in new_u2r)


In [259]:
#Building the classification models by using Decision Tree algorithm for all 4 databases(DOS,Probe,r2l,u2r) 
#For all features
import numpy as np
model_DOS = DecisionTreeClassifier()
model_DOS.fit(X_scaler_dos, y_train_dos.astype(int))

model_Probing = DecisionTreeClassifier()
model_Probing.fit(X_scaler_Probe, y_train_Probing.astype(int))

model_r2l =DecisionTreeClassifier()
model_r2l.fit(X_scaler_r2l,y_train_r2l.astype(int))

model_u2r = DecisionTreeClassifier()
model_u2r.fit(X_scaler_u2r, y_train_u2r.astype(int))


DecisionTreeClassifier()

In [260]:
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, 
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, 
                       random_state=0, splitter='best')

DecisionTreeClassifier(random_state=0)

In [261]:
#Prediction and Evaluation of the model for all attack types databases
#we are testing the model trained with the testing datasets(DOS,Probing, R2L, U2L) -all features
print('Decision tree classifier')
print()
print('Prediction evaluation for all features databases')
#Prediction 
y_DOS_pred = model_DOS.predict(X_test_scaler_dos)  #building confusion matrix (used in performance measurement) for all models
print('Predicted DOS attacks:', y_DOS_pred.sum())
print('Actual DOS attacks:', y_test_dos.sum())

y_Probing_pred = model_Probing.predict(X_test_scaler_probe)
print('Predicted Probing attacks:', y_Probing_pred.sum())
print('Actual Probing attacks:', y_probe_test.sum())

y_r2l_pred = model_r2l.predict(X_test_scaler_r2l)
print('Predicted r2l attacks:', y_r2l_pred.sum())
print('Actual r2l attacks:', y_test_r2l.sum())

y_u2r_pred = model_u2r.predict(X_test_scaler_u2r)
print('Predicted u2l attacks:', y_u2r_pred.sum())
print('Actual u2l attacks:', y_test_u2r.sum())




Decision tree classifier

Prediction evaluation for all features databases
Predicted DOS attacks: 23587
Actual DOS attacks: class    24631
dtype: int64
Predicted Probing attacks: 30340
Actual Probing attacks: class    16974
dtype: int64
Predicted r2l attacks: 12749
Actual r2l attacks: class    21251
dtype: int64
Predicted u2l attacks: 9810
Actual u2l attacks: class    10046
dtype: int64


In [262]:
#Crossvalidation: Accuracy, Precision, Recall ############################################################
print('Decision tree classifier')
print()
print('Accuracy, Precision, Recall values')
print()
#DOS attack
print('DOS attacks metrics')
accuracy = cross_val_score(model_DOS, X_test_scaler_dos, y_test_dos, cv=10, scoring='accuracy')
print('Model accuracy:', accuracy.mean(),'%')
precision = cross_val_score(model_DOS, X_test_scaler_dos, y_test_dos, cv =10, scoring='precision')
print('Model precision:', precision.mean(),'%')
recall = cross_val_score(model_DOS, X_test_scaler_dos, y_test_dos, scoring='recall', cv =10)
print('Model recall:', recall.mean(),'%')
f = cross_val_score(model_DOS, X_test_scaler_dos, y_test_dos, cv=10, scoring='f1')
print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))


#Probing
print()
print('Probing attacks metrics')
accuracy1 = cross_val_score(model_Probing, X_test_scaler_probe, y_probe_test, cv=10, scoring='accuracy')
print('Model accuracy:', accuracy1.mean(),'%')
precision1 = cross_val_score(model_Probing, X_test_scaler_probe, y_probe_test, cv =10, scoring='precision')
print('Model precision::', precision1.mean(),'%')
recall1 = cross_val_score(model_Probing, X_test_scaler_probe, y_probe_test, scoring='recall', cv =10)
print('Model recall:', recall1.mean(),'%')
f = cross_val_score(model_Probing, X_test_scaler_probe, y_probe_test, cv=10, scoring='f1')
print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))


#r2l attacks
print()
print('R2L attacks metrics')
accuracy2 = cross_val_score(model_r2l, X_test_scaler_r2l, y_test_r2l, cv=10, scoring='accuracy')
print('Model accuracy:', accuracy2.mean(),'%')
precision2 = cross_val_score(model_r2l, X_test_scaler_r2l, y_test_r2l, cv =10, scoring='precision')
print('Model precision::', precision2.mean(),'%')
recall2 = cross_val_score(model_r2l, X_test_scaler_r2l, y_test_r2l, scoring='recall', cv =10)
print('Model recall:', recall2.mean(),'%')
f = cross_val_score(model_r2l, X_test_scaler_r2l, y_test_r2l, cv=10, scoring='f1')
print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))


#U2R attacks
print()
print('U2R attacks metrics')
accuracy3 = cross_val_score(model_u2r, X_test_scaler_u2r , y_test_u2r, cv=10, scoring='accuracy')
print('Model accuracy:', accuracy3.mean(),'%')
precision3 = cross_val_score(model_u2r, X_test_scaler_u2r , y_test_u2r, cv =10, scoring='precision')
print('Model precision::', precision3.mean(),'%')
recall3 = cross_val_score(model_u2r, X_test_scaler_u2r , y_test_u2r, scoring='recall', cv =10)
print('Model recall:', recall3.mean(),'%')
f = cross_val_score(model_u2r, X_test_scaler_u2r , y_test_u2r, cv=10, scoring='f1')
print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))



Decision tree classifier

Accuracy, Precision, Recall values

DOS attacks metrics
Model accuracy: 0.9975540086365001 %
Model precision: 0.9984559390070222 %
Model recall: 0.9974256525664009 %
F-measure: 0.99773 (+/- 0.00236)

Probing attacks metrics
Model accuracy: 0.9980217739996823 %
Model precision:: 0.9982508151563338 %
Model recall: 0.9987642666124186 %
F-measure: 0.99871 (+/- 0.00083)

R2L attacks metrics
Model accuracy: 0.997062546490664 %
Model precision:: 0.9988669323051129 %
Model recall: 0.9974255466130966 %
F-measure: 0.99799 (+/- 0.00187)

U2R attacks metrics
Model accuracy: 0.9985682978442837 %
Model precision:: 0.9993827160493828 %
Model recall: 0.9994850669412976 %
F-measure: 0.99933 (+/- 0.00160)


In [ ]:
# K-Nearest Neighbors

#DOS
KN_model_DOS =  KNeighborsClassifier(n_neighbors=20)
KN_model_DOS.fit(X_scaler_dos, y_train_dos.astype(int))
#Probing
KN_model_Probing = KNeighborsClassifier(n_neighbors=20)
KN_model_Probing.fit(X_scaler_Probe, y_train_Probing.astype(int))
#r2l
KN_model_r2l = KNeighborsClassifier(n_neighbors=20)
KN_model_r2l.fit(X_scaler_r2l,y_train_r2l.astype(int))
#u2r
KN_model_u2r = KNeighborsClassifier(n_neighbors=20)
KN_model_u2r.fit(X_scaler_u2r, y_train_u2r.astype(int))




In [ ]:
#Prediction and Evaluation of the model for all attack types databases
#we are testing the model trained with the testing datasets(DOS,Probing, R2L, U2L) -all features
print(' K-Nearest Neighbors method ')
print()
print('Prediction evaluation for all features databases')
print()
#Prediction 
y_DOS_pred_KN = KN_model_DOS.predict(X_test_scaler_dos)  #building confusion matrix (used in performance measurement) for all models
print('Predicted DOS attacks:', y_DOS_pred_KN.sum())
print('Actual DOS attacks:', y_test_dos.sum())

y_Probing_pred_KN = KN_model_Probing.predict(X_test_scaler_probe)
print('Predicted Probing attacks:', y_Probing_pred_KN.sum())
print('Actual Probing attacks:', y_probe_test.sum())

y_r2l_pred_KN = KN_model_r2l.predict(X_test_scaler_r2l)
print('Predicted r2l attacks:', y_r2l_pred_KN.sum())
print('Actual r2l attacks:', y_test_r2l.sum())

y_u2r_pred_KN = KN_model_u2r.predict(X_test_scaler_u2r)
print('Predicted u2l attacks:', y_u2r_pred_KN.sum())
print('Actual u2l attacks:', y_test_u2r.sum())


In [ ]:
#Crossvalidation for K-Nearest Neighbors classification model: Accuracy, Precision, Recall
print(' K-Nearest Neighbors method' )
print()    
print('Accuracy, Precision, Recall values')
print()

#DOS attack
print('DOS attacks metrics')
accuracy = cross_val_score(KN_model_DOS, X_test_scaler_dos, y_test_dos, cv=10, scoring='accuracy')
print('Model accuracy:', accuracy.mean(),'%')
precision = cross_val_score(KN_model_DOS, X_test_scaler_dos, y_test_dos, cv =10, scoring='precision')
print('Model precision:', precision.mean(),'%')
recall = cross_val_score(KN_model_DOS, X_test_scaler_dos, y_test_dos, scoring='recall', cv =10)
print('Model recall:', recall.mean(),'%')
f = cross_val_score(KN_model_DOS, X_test_scaler_dos, y_test_dos, cv=10, scoring='f1')
print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))

#Probing
print()
print('Probing attacks metrics')
accuracy1 = cross_val_score(KN_model_Probing, X_test_scaler_probe, y_probe_test, cv=10, scoring='accuracy')
print('Model accuracy:', accuracy1.mean(),'%')
precision1 = cross_val_score(KN_model_Probing, X_test_scaler_probe, y_probe_test, cv =10, scoring='precision')
print('Model precision::', precision1.mean(),'%')
recall1 = cross_val_score(KN_model_Probing, X_test_scaler_probe, y_probe_test, scoring='recall', cv =10)
print('Model recall:', recall1.mean(),'%')
f = cross_val_score(KN_model_Probing, X_test_scaler_probe, y_probe_test, cv=10, scoring='f1')
print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))


#r2l attacks
print()
print('R2L attacks metrics')
accuracy2 = cross_val_score(KN_model_r2l, X_test_scaler_r2l, y_test_r2l, cv=10, scoring='accuracy')
print('Model accuracy:', accuracy2.mean(),'%')
precision2 = cross_val_score(KN_model_r2l, X_test_scaler_r2l, y_test_r2l, cv =10, scoring='precision')
print('Model precision::', precision2.mean(),'%')
recall2 = cross_val_score(KN_model_r2l, X_test_scaler_r2l, y_test_r2l, scoring='recall', cv =10)
print('Model recall:', recall2.mean(),'%')
f = cross_val_score(KN_model_r2l, X_test_scaler_r2l, y_test_r2l, cv=10, scoring='f1')
print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))


#U2R attacks
print()
print('U2R attacks metrics')
accuracy3 = cross_val_score(KN_model_u2r, X_test_scaler_u2r , y_test_u2r, cv=10, scoring='accuracy')
print('Model accuracy:', accuracy3.mean(),'%')
precision3 = cross_val_score(KN_model_u2r, X_test_scaler_u2r , y_test_u2r, cv =10, scoring='precision')
print('Model precision::', precision3.mean(),'%')
recall3 = cross_val_score(KN_model_u2r, X_test_scaler_u2r , y_test_u2r, scoring='recall', cv =10)
print('Model recall:', recall3.mean(),'%')
f = cross_val_score(KN_model_u2r, X_test_scaler_u2r , y_test_u2r, cv=10, scoring='f1')
print("F-measure: %0.5f (+/- %0.5f)" % (f.mean(), f.std() * 2))
